<a href="https://colab.research.google.com/github/TarteTatin1212/contents_search/blob/main/contents_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#モジュールのインストール
!pip install langid

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langid: filename=langid-1.1.6-py3-none-any.whl size=1941173 sha256=c922bffaf043a7906273d84bc5e8ab914963e642d4d5ae3811dee9669e07107f
  Stored in directory: /root/.cache/pip/wheels/23/c8/c6/eed80894918490a175677414d40bd7c851413bbe03d4856c3c
Successfully built langid


In [ ]:
import langid              # 言語識別のためのライブラリ
import pandas as pd        # データ分析のためのライブラリ（データフレームを操作するため）
import json               # JSON形式のデータを操作するためのモジュール
import pytz               # タイムゾーン情報を扱うためのライブラリ
from datetime import datetime, timedelta # 日付と時間を操作するためのモジュール
from apiclient.discovery import build    # Google APIクライアントを構築するためのモジュール
from google.colab import drive           # Google Colab環境でGoogle Driveを操作するためのモジュール
from datetime import datetime
import requests


In [ ]:
# Google Driveをマウント
drive.mount('/content/drive')

# ディレクトリをSenior_Projectに変更
%cd '/content/drive/MyDrive/Horita_Semi/Senior_Project'

Mounted at /content/drive
/content/drive/MyDrive/Horita_Semi/Senior_Project


In [ ]:
%ls

2023年6月1日課題点羅列.gdoc  youtube_data_japanese.csv
api_key.txt                  youtube_data_japanese.json
contents_search.ipynb        youtube_data_other.csv
face_recognition.ipynb       youtube_data_other.json


In [ ]:
def read_api_key(filepath):
    """APIキーをファイルから読み込む"""
    with open(filepath, 'r') as file:  # 指定されたファイルパスを読み取りモードで開く
        return file.readline().strip() # ファイルの最初の行を読み取り、余分な空白や改行を除去して返す


In [ ]:
def build_youtube_client(api_key):
    """YouTube APIクライアントを初期化"""
    return build("youtube", "v3", developerKey=api_key)  # Google APIのbuild関数を使用してYouTube APIのクライアントをv3バージョンで初期化し、それを返す


In [ ]:
def get_search_period():
    """検索期間を取得する関数"""
    current_time_japan = datetime.now(pytz.timezone("Asia/Tokyo"))  # 現在の日本時間を取得

    # 前日の日付を取得
    previous_day = current_time_japan - timedelta(days=1)

    # 前日の18:00と21:00を取得
    start_time = previous_day.replace(hour=18, minute=0, second=0, microsecond=0)
    end_time = previous_day.replace(hour=21, minute=0, second=0, microsecond=0)

    return start_time.isoformat(), end_time.isoformat()  # 開始時刻と終了時刻をISO 8601形式の文字列として返す


In [ ]:
def print_search_period_dates():
    """検索期間の日付を出力する関数"""
    start_time, _ = get_search_period()  # 検索期間の開始時刻を取得

    # ISO形式の文字列から日付と時刻を取得
    start_datetime = datetime.fromisoformat(start_time).astimezone(pytz.timezone("Asia/Tokyo"))

    # 日付をフォーマットして出力
    print(f"Search date: {start_datetime.strftime('%Y-%m-%d')} (Japan Time)")


In [ ]:
def fetch_videos(youtube, start_time_iso, end_time_iso):
    """指定された期間にアップロードされた動画を検索する関数"""
    # YouTubeのsearchリソースのlistメソッドを用いて動画検索を実行
    return youtube.search().list(
        q="",                   # 検索クエリ（空文字列で全ての動画を対象とする）
        part="id",              # 必須パラメータ。取得するリソースの部分を指定。ここでは動画のIDのみを取得
        type="video",           # 検索するリソースのタイプを指定。ここでは動画のみを対象とする
        publishedAfter=start_time_iso,  # 指定された時刻以降に公開された動画のみを対象とする
        maxResults=100,         # 一度のリクエストで取得する結果の最大数を指定。最大値は50だが、例として100が示されている（エラーになる可能性あり）
        regionCode="JP",        # 公開地域を指定。ここでは日本で公開された動画のみを対象とする
    ).execute()                # 検索リクエストを実行して結果を取得


In [ ]:
def print_fetched_videos(youtube, start_time_iso, end_time_iso):
    """指定された期間にアップロードされた動画の情報を出力する関数"""

    # fetch_videos関数で動画情報を取得
    search_response = fetch_videos(youtube, start_time_iso, end_time_iso)

    # 検索結果の動画情報を一つずつ出力
    for item in search_response.get("items", []):
        video_id = item["id"]["videoId"]
        print(f"Video ID: {video_id}")

    print(f"Total videos fetched: {len(search_response.get('items', []))}")


In [ ]:
def classify_and_store_videos(search_response, youtube):
    """動画を日本語とその他のカテゴリに分類する関数"""

    # 日本語の動画を保存するリスト
    data_japanese = []
    # その他の言語の動画を保存するリスト
    data_other = []

    # 検索応答から各動画の情報を取得
    for item in search_response["items"]:
        # 動画のIDを取得
        video_id = item["id"]["videoId"]

        # 動画の詳細情報をYouTube APIから取得
        video_response = youtube.videos().list(part="snippet,statistics", id=video_id).execute()

        # 取得した動画の詳細情報を処理
        for item in video_response["items"]:
            # 動画のタイトルを取得
            title = item["snippet"]["title"]

            # タイトルの言語を判定
            lang, _ = langid.classify(title)

            # チャンネルのタイトルを取得
            channel_title = item["snippet"]["channelTitle"]
            # サムネイルのURLを取得
            thumbnail_url = item["snippet"]["thumbnails"]["default"]["url"]
            # 視聴回数を取得
            view_count = item["statistics"]["viewCount"]

            # 上記で取得した情報を辞書にまとめる
            video_info = {
                "title": title,
                "channelTitle": channel_title,
                "thumbnailUrl": thumbnail_url,
                "viewCount": view_count
            }

            # 言語が日本語であればdata_japaneseに、それ以外であればdata_otherに追加
            if lang == 'ja':
                data_japanese.append(video_info)
            else:
                data_other.append(video_info)

    # 2つのリストを返す
    return data_japanese, data_other


In [ ]:
def print_video_data(data_japanese, data_other):
    """辞書に格納された動画データをターミナルに出力する関数"""

    print("\n=== 日本語の動画 ===")
    for video in data_japanese:
        print(f"タイトル: {video['title']}")
        print(f"チャンネル: {video['channelTitle']}")
        print(f"サムネイルURL: {video['thumbnailUrl']}")
        print(f"視聴回数: {video['viewCount']}")
        print("---------------------------")

    print("\n=== その他の言語の動画 ===")
    for video in data_other:
        print(f"タイトル: {video['title']}")
        print(f"チャンネル: {video['channelTitle']}")
        print(f"サムネイルURL: {video['thumbnailUrl']}")
        print(f"視聴回数: {video['viewCount']}")
        print("---------------------------")

In [ ]:
def save_data_to_files(data_japanese, data_other):
    """データをJSONおよびCSVファイルに保存する関数"""
    # 現在の日付と時間を取得
    current_datetime = datetime.now().strftime('%Y%m%d_%H%M%S')

    # 日本語の動画データをJSONファイルとして保存
    with open(f'youtube_data_japanese_{current_datetime}.json', 'w') as json_file:
        json.dump(data_japanese, json_file)

    # 各日本語動画データのサムネイルをjpgファイルとして保存
    for i, video in enumerate(data_japanese):
        thumbnail_url = video['thumbnailUrl']
        response = requests.get(thumbnail_url)
        if response.status_code == 200:  # ステータスコード200はリクエストが成功したことを示す
            with open(f'thumbnail_{i}_{current_datetime}.jpg', 'wb') as img_file:
                img_file.write(response.content)

    # その他の言語の動画データをJSONファイルとして保存
    with open(f'youtube_data_other_{current_datetime}.json', 'w') as json_file:
        json.dump(data_other, json_file)

    # 日本語の動画データをpandasのDataFrameオブジェクトに変換
    df_japanese = pd.DataFrame(data_japanese)
    df_japanese.to_csv(f'youtube_data_japanese_{current_datetime}.csv', index=False)

    # その他の言語の動画データをpandasのDataFrameオブジェクトに変換
    df_other = pd.DataFrame(data_other)
    df_other.to_csv(f'youtube_data_other_{current_datetime}.csv', index=False)

In [ ]:
# 上記の関数を使用して、処理を順番に実行する

# 'api_key.txt' ファイルからAPIキーを読み込む
api_key = read_api_key('/content/drive/MyDrive/Horita_Semi/Senior_Project/api_key.txt')

# 読み込んだAPIキーを使ってYouTube APIクライアントを初期化
youtube = build_youtube_client(api_key)

# 日本時間で18:00から21:00までのISO 8601形式の開始時刻と終了時刻を取得
start_time_iso, end_time_iso = get_search_period()

# 検索日を出力
print_search_period_dates()

# 上で取得した期間内にアップロードされた動画をYouTube APIを使用して検索
search_response = fetch_videos(youtube, start_time_iso, end_time_iso)

# 動画情報を出力
print_fetched_videos(youtube, start_time_iso, end_time_iso)

# 検索結果から動画のタイトルをもとに言語を識別し、日本語とその他に分類
data_japanese, data_other = classify_and_store_videos(search_response, youtube)

# データをターミナルに出力
print_video_data(data_japanese, data_other)

# 分類された動画データをJSONおよびCSVファイルに保存
save_data_to_files(data_japanese, data_other)


Search date: 2023-09-28 (Japan Time)
Video ID: SydrJQ6DeMg
Video ID: nLgt4Z5L8XI
Video ID: IqwIOlhfCak
Video ID: dx0ksJZjZic
Video ID: c6kBdzjTjNY
Video ID: TdLKOoJ5Rrs
Video ID: H35GdCASqbU
Video ID: unLbIM8k3s8
Video ID: 8xVCKwlj42c
Video ID: vq-nvJ6q8bI
Video ID: lpZUNQJ6-BU
Video ID: RgdOJv-rcuU
Video ID: 2j3iwRnlH1g
Video ID: 582KZ_aFJVE
Video ID: wCERwslAl68
Video ID: ucgxrQ1iBow
Video ID: awKJVhHxqiQ
Video ID: swNMx6LPDpw
Video ID: sqQTr0y55_E
Video ID: Gx6VeIIbWwM
Video ID: AdyiQYaXeBs
Video ID: 5Y18UGWN4Yc
Video ID: _EcyGYnySPY
Video ID: h58mobFgHOE
Video ID: V9ocBsADEMs
Video ID: mN3-Lpsaav4
Video ID: ud7-N4VyE84
Video ID: xsoi5claak8
Video ID: zX7d8_csySA
Video ID: DRPjywCkqYc
Video ID: CeKY3kV9kqU
Video ID: 5EaobKJ5kIk
Video ID: Z8VbXVQLiDA
Video ID: NjJdtalUTdY
Video ID: LIDGbLEo9qE
Video ID: i6z-fRoVcfE
Video ID: _43SaPADGmQ
Video ID: ET_BMYwVH3Y
Video ID: 9VcAWF19z2s
Video ID: bB3Wrv8TSe4
Video ID: ryuQx_V0gsA
Video ID: 9qoakka2BDY
Video ID: HfRB-oa_EWY
Video ID: 8n6s3GK